# Toronto Neighbourhoods assessment

### Import and install libraries

In [7]:
import pandas as pd
import numpy as np

print('imported libraries')

imported libraries


In [8]:
!pip install bs4
#!pip install requests

print('b4 installed')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
b4 installed


In [9]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
print('imported requests')

imported requests


# Invoke URL request

In [10]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)

# Get HTML content and check

In [11]:
html_Content=r.text
#html_Content

# Use BeautifulSoup to parse html content

In [12]:
soup = BeautifulSoup(html_Content, 'html5lib')
#print(soup.prettify())

In [13]:
tag_object=soup.title
print("tag object:",tag_object)

tag object: <title>List of postal codes of Canada: M - Wikipedia</title>


# Get the required table

In [14]:
table=soup.find('table')
#table

# Logic to read the table elements and convret to dataframe and check

In [16]:
rows=[]
for row in table.findAll('td'):
    text=row.p.text
    if('\n' in text):
        text=text.replace('\n','')
    #print('text=',text)
    code=text[0:3]
    name=row.span.text               
    name=name.replace(' /',',')              
    if('(' in name):
        borough=name.split('(')[0]
        Name=name.split('(')[1].replace(')','')
    else:
        borough=name
        Name=borough  
        #Ignore Not Assigned Boroughs (Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.)    
    if('Not assigned' not in borough):
        #print(code,borough,Name)
        cell1={}
        if(code not in cell1):            
            cell1['PostalCode']=code
            cell1['Borough']=borough
            #If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
            if('Not Assigned' in Name):
                cell1['Neighbourhood']=borough
            else:
                cell1['Neighbourhood']=Name
            #print('cell1=',cell1)
            rows.append(cell1)
            #print(rows)
        #else:
            #cell1['PostalCode']=code
            #cell1['Borough'].append(Borough)      
            #cell1['Name'].append(Name)
#print(rows)                
df=pd.DataFrame(rows)
df


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Use the replacement logic (given) and analyse the dataframe

In [20]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [21]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [22]:
df.describe()

,PostalCode,Borough,Neighbourhood
count,103,103,103
unique,103,15,103
top,M1R,North York,Berczy Park
freq,1,24,1


In [23]:
df.shape

(103, 3)